In [48]:
import pandas as pd
from tqdm import tqdm

## Poster Confusion Matrix

Analyzing Results of 13,941 Patients

In [109]:
df = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\APOE-SLAT\Modeling\Storage\data\20K_sample_8_11.csv")
df2 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\APOE-SLAT\Baseline_Model\data\tanish_predictions_with_structured_features.csv")

In [78]:
df2["patient_CI"] = p_CI

In [110]:
e2 = df2[(df2["APOE"] == "e2/e2") | (df2["APOE"] == "e2/e3")]
e3 = df2[df2["APOE"] == "e3/e3"]
e4 = df2[(df2["APOE"] == "e2/e4 or e1/e3") | (df2["APOE"] == "e3/e4") | (df2["APOE"] == "e4/e4")]

In [111]:
(len(e2) + len(e3) + len(e4)) - len(df2)

0

In [112]:
df2["APOE"].value_counts()

e3/e3             8751
e3/e4             2862
e2/e3             1675
e2/e4 or e1/e3     302
e4/e4              272
e2/e2               79
Name: APOE, dtype: int64

In [113]:
len(e2), len(e3), len(e4)

(1754, 8751, 3436)

In [114]:
e2["AD_Med_or_ICD_Code"].value_counts()

0    1562
1     192
Name: AD_Med_or_ICD_Code, dtype: int64

In [115]:
e3["AD_Med_or_ICD_Code"].value_counts()

0    7771
1     980
Name: AD_Med_or_ICD_Code, dtype: int64

In [116]:
e4["AD_Med_or_ICD_Code"].value_counts()

0    2859
1     577
Name: AD_Med_or_ICD_Code, dtype: int64

In [117]:
e2["patient_CI"].value_counts()

1    1080
0     674
Name: patient_CI, dtype: int64

In [118]:
e3["patient_CI"].value_counts()

1    5415
0    3336
Name: patient_CI, dtype: int64

In [119]:
e4["patient_CI"].value_counts()

1    2169
0    1267
Name: patient_CI, dtype: int64

In [126]:
len(df2[(df2["patient_CI"] == 1) & (df2["AD_Med_or_ICD_Code"] == 1)])

1665

In [127]:
len(df2[(df2["patient_CI"] == 1) & (df2["AD_Med_or_ICD_Code"] == 0)])

6999

In [128]:
len(df2[(df2["patient_CI"] == 0) & (df2["AD_Med_or_ICD_Code"] == 1)])

84

In [129]:
len(df2[(df2["patient_CI"] == 0) & (df2["AD_Med_or_ICD_Code"] == 0)])

5193

## Analyzing False Positives
Model Predicts CI, no prescence of Med/ICD Code <br>
What portion of these are where models if correct, and what portion is where model is wrong

In [131]:
predictions = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\APOE-SLAT\Baseline_Model\data\tanish_predictions_with_structured_features.csv")

In [134]:
FP = predictions[(predictions["patient_CI"] == 1) & (predictions["AD_Med_or_ICD_Code"] == 0)]

In [136]:
sample_set = FP["patient_id"].to_list()[:50]

In [138]:
dataset = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\APOE-SLAT\nlp_annotator-dev\app\load_data\SLAT_production_7_24.csv")

C:\Users\MIND_DS\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [271]:
pred_proba = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\APOE-SLAT\Modeling\Storage\Predictions\lr_binary_classification_8_17.csv")

In [264]:
sampling_df = pd.DataFrame()
pred = []
proba = []

for i in tqdm(range(len(sample_set))):
    sampling_df = sampling_df.append(dataset[dataset["PatientID"] == sample_set[i]])
    pred += (pred_proba[pred_proba["patient_id"] == sample_set[i]]["predictions"].to_list())
    proba += (pred_proba[pred_proba["patient_id"] == sample_set[i]]["probability"].to_list())

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [00:03<00:00, 13.17it/s]


In [268]:
sampling_df["predictions"] = pred
sampling_df["probability"] = proba

In [270]:
sampling_df.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\APOE-SLAT\Modeling\Storage\Analysis\50_FP_sample.csv", index = False)

## Fixing Error of Sequences not being padded

In [145]:
not_padded = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\FINAL\not_padded_matches_7_14.csv")

In [147]:
padded = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\FINAL\padded_matches_7_14.csv")

C:\Users\MIND_DS\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [151]:
len(padded[padded["padded_merged_regex_sent"].str.len() < 500])

3339

In [166]:
not_padded.columns = ['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'Unnamed: 0.1.1.1',
       'EMPI', 'MRN', 'PatientID', 'PatientEncounterID', 'ContactDTS',
       'EncounterTypeDSC', 'NoteID', 'InpatientNoteTypeDSC', 'buckets',
       'NoteCSNID', 'LineNBR', 'NoteTXT', 'regex_location', 'regex_sent',
       'regex_match', 'match_count', 'pruned_regex_location',
       'merged_row_location', 'padded_merged_regex_sent', 'merged_regex_match',
       'sequence_level_regex_location', 'sequence_level_regex_match',
       'char_count', 'note_char_count']

In [168]:
entire_matches = pd.concat([padded, not_padded])

In [172]:
len(entire_matches[entire_matches["padded_merged_regex_sent"].str.len() < 500])

3339

In [204]:
sequences = []
counter = 0

for index, row in dataset.iterrows():
    match = entire_matches[entire_matches["NoteID"] == row["NoteID"]]
    
    if (len(match) > 1):
        match = match[match["NoteTXT"] == str(row["NoteTXT"])]
        if (len(match) > 1):
            print("Issue")
            break
        sequences.append(match["padded_merged_regex_sent"].values[0])
    else:
        sequences.append(match["padded_merged_regex_sent"].values[0])
        
    if (len(sequences) == counter):
        print("Issue")
        break
        
    counter += 1
    if (counter % 100000 == 0):
        print("Finished ", counter, " sequences", len(sequences))

Finished  100000  sequences 100000
Finished  200000  sequences 200000


In [207]:
len(sequences)

279224

In [ ]:
# dataset["regex_sent"] = sequences

In [208]:
from sqlite3 import connect
conn = connect(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\APOE-SLAT\nlp_annotator-dev\app\db.sqlite3")
gui_sentence = pd.read_sql("SELECT * FROM gui_sentence", conn)

In [223]:
new = pd.DataFrame()
new["id"] = gui_sentence["id"]
new["Contents"] = sequences
new["Note_id"] = gui_sentence["Note_id"]

In [226]:
new["Contents"] = new["Contents"].apply(json.dumps)

In [227]:
import json

def is_json(myjson):
    try:
        json_object = json.loads(myjson)
    except ValueError as e:
        return False
    return True

is_json(new.loc[7658]["Contents"])

True

In [230]:
new.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\APOE-SLAT\nlp_annotator-dev\app\slat_8_16.csv", index = False, header = False)

In [232]:
dataset["regex_sent"] = new["Contents"]

In [234]:
dataset.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\SLAT\SLAT_8_17.csv", index = False)